<a href="https://colab.research.google.com/github/dharshinivenkatesan/pyspark/blob/main/Skill_Lab_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import col,greatest,lit,expr,regexp_replace,sum,round,count,avg,countDistinct,max,when
from pyspark.sql.window import Window
from datetime import datetime, timedelta

In [2]:
spark = SparkSession.builder.appName("Hamadullah").getOrCreate()

In [4]:
#Code to read data into pd dataframe from excel sheet and convert to spark dataframe
'''file_path = "path"
data_pd = pd.read_excel(file_path, sheet_name="orders")
data_df = spark.createDataFrame(data_pd)'''

'file_path = "path"\ndata_pd = pd.read_excel(file_path, sheet_name="orders")\ndata_df = spark.createDataFrame(data_pd)'

#Question 1

In [5]:

data = [
    (1, "Titanium Cartridge", 2500),
    (2, "Hand Grenade", 1800),
    (3, "AK47", 1200),
    (4, "Timer", 900),
    (5, "RDX", 800),
    (6, "Battery", 600),
    (7, "AK47", 400),
    (8, "Insulated Wire", 1500),
    (9, "RDX", 700),
    (10, "Glock", 1300),
    (11, "Kevlar", 300),
    (12, "Mine", 1000),
    (13, "Helmet", 400),
    (14, "Gloves", 300),
    (15, "Paper", 1300),
    (16, "RDX", 900),
    (17, "Sulphur", 1000),
    (18, "Mine", 500)
]

columns = ["sale_id", "product_name", "sales"]

data_df = spark.createDataFrame(data, schema=columns)

result = (
    data_df.groupBy("product_name")
    .sum("sales")
    .withColumnRenamed("sum(sales)", "total_sales")
    .orderBy(col("total_sales").desc())
    .limit(5)
)

result.show()

+------------------+-----------+
|      product_name|total_sales|
+------------------+-----------+
|Titanium Cartridge|       2500|
|               RDX|       2400|
|      Hand Grenade|       1800|
|              AK47|       1600|
|    Insulated Wire|       1500|
+------------------+-----------+



#Question 2

In [6]:
import random


data = [(i, random.randint(50, 100), random.randint(50, 100), random.randint(50, 100), random.randint(50, 100)) for i in range(1, 21)]
columns = ["employee_id", "quarter1", "quarter2", "quarter3", "quarter4"]

data_df = spark.createDataFrame(data, columns)
result = data_df.withColumn("max_score", greatest("quarter1", "quarter2", "quarter3", "quarter4"))
#result = result.drop("quarter1", "quarter2", "quarter3", "quarter4")

result.show()

+-----------+--------+--------+--------+--------+---------+
|employee_id|quarter1|quarter2|quarter3|quarter4|max_score|
+-----------+--------+--------+--------+--------+---------+
|          1|      95|      95|      50|     100|      100|
|          2|      52|      52|      80|      76|       80|
|          3|      60|      74|      56|      93|       93|
|          4|      94|      57|      68|      75|       94|
|          5|      63|      61|      71|      69|       71|
|          6|      58|      85|      66|      74|       85|
|          7|      99|      89|      71|      89|       99|
|          8|      67|      59|      59|      61|       67|
|          9|      61|      75|      92|      81|       92|
|         10|     100|      87|      82|      62|      100|
|         11|      60|      51|      80|      78|       80|
|         12|      58|      50|      53|      75|       75|
|         13|      85|      69|      53|      96|       96|
|         14|      95|      51|      71|

#Question 3

In [7]:
data = [(i, random.randint(100, 10000)) for i in range(1, 31)]
columns = ["customer_id", "spending"]

data_df = spark.createDataFrame(data, columns)

result = data_df.withColumn("currency", lit("USD"))

result.show()

+-----------+--------+--------+
|customer_id|spending|currency|
+-----------+--------+--------+
|          1|    4480|     USD|
|          2|    5436|     USD|
|          3|    8824|     USD|
|          4|    4364|     USD|
|          5|    7437|     USD|
|          6|    4645|     USD|
|          7|    1945|     USD|
|          8|    9635|     USD|
|          9|    1969|     USD|
|         10|    7865|     USD|
|         11|    7028|     USD|
|         12|    5890|     USD|
|         13|    5891|     USD|
|         14|    8423|     USD|
|         15|    4063|     USD|
|         16|    4324|     USD|
|         17|    3546|     USD|
|         18|    5264|     USD|
|         19|    8590|     USD|
|         20|    1641|     USD|
+-----------+--------+--------+
only showing top 20 rows



#Question 4

In [8]:
data = [
    ("Cartridge", "Titanium", 1500),
    ("Cartridge", "Steel", 1800),
    ("Cartridge", "Brass", 1200),
    ("Gun", "AK47", 900),
    ("Explosive", "RDX", 800),
    ("Supply", "Battery", 600),
    ("Gun", "AK47", 400),
    ("Supply", "Insulated Wire", 1500),
    ("Explosive", "RDX", 700),
    ("Gun", "Glock", 1300),
    ("Supply", "Kevlar", 300),
    ("Explosive", "Mine", 1000),
    ("Supply", "Helmet", 400),
    ("Supply", "Gloves", 300),
    ("Supply", "Paper", 1300),
    ("Explosive", "RDX", 900),
    ("Explosive", "Sulphur", 1000),
    ("Explosive", "Mine", 500)
]

columns = ["category", "subcategory", "sales"]

data_df = spark.createDataFrame(data, columns)

data_df.createOrReplaceTempView("sales_view")


result = spark.sql(
    """
    SELECT category, subcategory, SUM(sales) AS total_sales
    FROM sales_view
    GROUP BY category, subcategory
    ORDER BY category, subcategory
    """
)
result.show()

result = spark.sql(
    """
    SELECT category, SUM(sales) AS total_sales
    FROM sales_view
    GROUP BY category
    ORDER BY category
    """
)

result.show()

result = spark.sql(
    """
    SELECT subcategory, SUM(sales) AS total_sales
    FROM sales_view
    GROUP BY subcategory
    ORDER BY subcategory
    """
)



result.show()

+---------+--------------+-----------+
| category|   subcategory|total_sales|
+---------+--------------+-----------+
|Cartridge|         Brass|       1200|
|Cartridge|         Steel|       1800|
|Cartridge|      Titanium|       1500|
|Explosive|          Mine|       1500|
|Explosive|           RDX|       2400|
|Explosive|       Sulphur|       1000|
|      Gun|          AK47|       1300|
|      Gun|         Glock|       1300|
|   Supply|       Battery|        600|
|   Supply|        Gloves|        300|
|   Supply|        Helmet|        400|
|   Supply|Insulated Wire|       1500|
|   Supply|        Kevlar|        300|
|   Supply|         Paper|       1300|
+---------+--------------+-----------+

+---------+-----------+
| category|total_sales|
+---------+-----------+
|Cartridge|       4500|
|Explosive|       4900|
|      Gun|       2600|
|   Supply|       4400|
+---------+-----------+

+--------------+-----------+
|   subcategory|total_sales|
+--------------+-----------+
|          AK47| 

#Question 5

In [9]:
data = [(i, random.randint(50, 500), f"{random.uniform(5, 30):.2f}%") for i in range(1, 21)]
columns = ["product_id", "price", "discount_percent"]


data_df = spark.createDataFrame(data, columns)

data_df = data_df.withColumn(
    "discount_percent", regexp_replace("discount_percent", "%", "").cast("double")
)


result = data_df.withColumn(
    "final_price", expr("price - (price * discount_percent / 100)")
)


result.show()


+----------+-----+----------------+------------------+
|product_id|price|discount_percent|       final_price|
+----------+-----+----------------+------------------+
|         1|  198|            7.28|          183.5856|
|         2|  476|           17.22|          394.0328|
|         3|  476|           17.43|393.03319999999997|
|         4|   86|            29.7|            60.458|
|         5|  346|           14.71|          295.1034|
|         6|   50|           22.99|            38.505|
|         7|  134|           13.93|          115.3338|
|         8|  280|            7.89|           257.908|
|         9|  235|           18.74|           190.961|
|        10|  382|           17.33|          315.7994|
|        11|  459|           11.64|          405.5724|
|        12|  133|           29.36|           93.9512|
|        13|   90|           28.34|            64.494|
|        14|  362|            8.85|           329.963|
|        15|  483|           21.78|          377.8026|
|        1

#Question 6

In [10]:
data = [
    (f"Product_{random.randint(1, 10)}", f"Region_{random.randint(1, 5)}", i, random.randint(1, 5), f"Comment_{i}")
    for i in range(1, 101)
]
columns = ["product", "region", "feedback_id", "rating", "comments"]


data_df = spark.createDataFrame(data, columns)


result = data_df.filter(col("rating").isin(4, 5))
scalar = result.count()

result.show()


print(f"Number of feedback entries with high ratings (4 or 5): {scalar}")


+----------+--------+-----------+------+----------+
|   product|  region|feedback_id|rating|  comments|
+----------+--------+-----------+------+----------+
| Product_2|Region_4|          4|     5| Comment_4|
| Product_4|Region_3|          5|     4| Comment_5|
| Product_1|Region_5|         11|     4|Comment_11|
| Product_6|Region_4|         13|     5|Comment_13|
| Product_8|Region_5|         14|     4|Comment_14|
| Product_6|Region_4|         16|     5|Comment_16|
|Product_10|Region_5|         18|     5|Comment_18|
| Product_1|Region_2|         22|     4|Comment_22|
| Product_4|Region_4|         30|     4|Comment_30|
| Product_1|Region_1|         31|     5|Comment_31|
| Product_9|Region_3|         34|     4|Comment_34|
| Product_7|Region_3|         36|     5|Comment_36|
| Product_4|Region_1|         37|     4|Comment_37|
|Product_10|Region_3|         38|     4|Comment_38|
| Product_6|Region_2|         39|     4|Comment_39|
| Product_4|Region_4|         40|     5|Comment_40|
| Product_8|

#Question 7

In [11]:
data = [
    (f"Product_{random.randint(1, 5)}", (datetime(2024, 1, 1) + timedelta(days=random.randint(0, 30))).strftime('%Y-%m-%d'), random.randint(100, 500))
    for _ in range(100)
]
columns = ["product_id", "date", "sales"]


data_df = spark.createDataFrame(data, columns)


window_spec = Window.partitionBy("product_id").orderBy("date").rowsBetween(Window.unboundedPreceding, Window.currentRow)


result = data_df.withColumn("running_total", sum("sales").over(window_spec))


result.show()

+----------+----------+-----+-------------+
|product_id|      date|sales|running_total|
+----------+----------+-----+-------------+
| Product_1|2024-01-01|  140|          140|
| Product_1|2024-01-01|  294|          434|
| Product_1|2024-01-02|  271|          705|
| Product_1|2024-01-02|  184|          889|
| Product_1|2024-01-05|  281|         1170|
| Product_1|2024-01-06|  266|         1436|
| Product_1|2024-01-06|  377|         1813|
| Product_1|2024-01-08|  362|         2175|
| Product_1|2024-01-08|  256|         2431|
| Product_1|2024-01-13|  410|         2841|
| Product_1|2024-01-13|  346|         3187|
| Product_1|2024-01-14|  430|         3617|
| Product_1|2024-01-16|  116|         3733|
| Product_1|2024-01-20|  318|         4051|
| Product_1|2024-01-24|  159|         4210|
| Product_1|2024-01-26|  498|         4708|
| Product_1|2024-01-28|  172|         4880|
| Product_2|2024-01-03|  327|          327|
| Product_2|2024-01-07|  237|          564|
| Product_2|2024-01-10|  208|   

#Question 8

i am not sure what this question wants us to do

In [12]:
data = [
    ("john.doe@gmail.com", "gmail.com"),
    ("jane.smith@yahoo.com", "yahoo.com"),
    ("adam.west@gmail.com", "gmail.com"),
    ("eve.jackson@outlook.com", "outlook.com")
]


columns = ["email", "domain"]


data_df = spark.createDataFrame(data, columns)


result = data_df.filter(col("email").like("%gmail%"))


result = result.withColumn("extracted_domain", expr("substr(domain, 1, length(domain))"))


result.show()


+-------------------+---------+----------------+
|              email|   domain|extracted_domain|
+-------------------+---------+----------------+
| john.doe@gmail.com|gmail.com|       gmail.com|
|adam.west@gmail.com|gmail.com|       gmail.com|
+-------------------+---------+----------------+



#Question 9

In [13]:
data = [
    (1, 100.456, 5.234),
    (2, 200.789, 10.567),
    (3, 50.111, 2.345),
    (4, 300.523, 15.678)
]


columns = ["transaction_id", "amount", "tax"]


data_df = spark.createDataFrame(data, columns)


result = data_df.withColumn("total_amount", col("amount") + col("tax"))


result = result.withColumn("total_amount_rounded", round(col("total_amount"), 2))


result.show()

+--------------+-------+------+------------------+--------------------+
|transaction_id| amount|   tax|      total_amount|total_amount_rounded|
+--------------+-------+------+------------------+--------------------+
|             1|100.456| 5.234|            105.69|              105.69|
|             2|200.789|10.567|           211.356|              211.36|
|             3| 50.111| 2.345|52.455999999999996|               52.46|
|             4|300.523|15.678|           316.201|               316.2|
+--------------+-------+------+------------------+--------------------+



#Question 10

In [14]:
data = [
    (1, "2024-12-01", 150.50),
    (2, "2024-12-02", 50.00),
    (1, "2024-12-03", 200.75),
    (3, "2024-12-04", 300.20),
    (2, "2024-12-05", 120.00),
    (3, "2024-12-06", 95.00)
]


columns = ["customer_id", "order_date", "amount"]


data_df = spark.createDataFrame(data, columns)


data_df.createOrReplaceTempView("customer_orders")


query = """
SELECT
    customer_id,
    AVG(amount) AS average_order_amount
FROM
    customer_orders
GROUP BY
    customer_id
HAVING
    AVG(amount) > 100
"""
result = spark.sql(query)


result.show()

+-----------+--------------------+
|customer_id|average_order_amount|
+-----------+--------------------+
|          1|             175.625|
|          3|               197.6|
+-----------+--------------------+



#Question 11

In [15]:
data = [
    (1, "A", 500),
    (1, "B", 300),
    (2, "A", 700),
    (2, "C", 200),
    (3, "B", 450),
    (3, "C", 550)
]


columns = ["store_id", "product_id", "sales"]


data_df = spark.createDataFrame(data, columns)


result = data_df.groupBy("store_id").agg(sum("sales").alias("total_sales"))


result.show()

+--------+-----------+
|store_id|total_sales|
+--------+-----------+
|       1|        800|
|       2|        900|
|       3|       1000|
+--------+-----------+



#Question 12

In [16]:
data = [
    ("HR", "Manager", 1),
    ("HR", "Executive", 2),
    ("IT", "Manager", 3),
    ("IT", "Developer", 4),
    ("Finance", "Analyst", 5),
    ("Finance", "Manager", 6),
    ("IT", "Developer", 7),
    ("HR", "Executive", 8)
]


columns = ["department", "designation", "employee_id"]


data_df = spark.createDataFrame(data, columns)


result = data_df.groupBy("department", "designation").agg(count("employee_id").alias("employee_count"))


result.show()

+----------+-----------+--------------+
|department|designation|employee_count|
+----------+-----------+--------------+
|        IT|  Developer|             2|
|        HR|    Manager|             1|
|        IT|    Manager|             1|
|        HR|  Executive|             2|
|   Finance|    Manager|             1|
|   Finance|    Analyst|             1|
+----------+-----------+--------------+



#Question 13

In [17]:
data = [
    (1, "North", 150.50),
    (2, "South", 200.75),
    (3, "North", 100.00),
    (4, "East", 250.30),
    (5, "West", 300.00),
    (6, "South", 180.20)
]


columns = ["customer_id", "region", "purchase_amount"]


data_df = spark.createDataFrame(data, columns)


result = data_df.groupBy("region").agg(avg("purchase_amount").alias("average_purchase_amount"))


result.show()

+------+-----------------------+
|region|average_purchase_amount|
+------+-----------------------+
| South|                190.475|
| North|                 125.25|
|  East|                  250.3|
|  West|                  300.0|
+------+-----------------------+



#Question 14

In [18]:
data = [
    ("Electronics", "Mobile", 500),
    ("Electronics", "Laptop", 1200),
    ("Electronics", "Tablet", 300),
    ("Furniture", "Chair", 150),
    ("Furniture", "Table", 300),
    ("Clothing", "Shirts", 100),
    ("Clothing", "Pants", 200)
]


columns = ["category", "subcategory", "sales"]


data_df = spark.createDataFrame(data, columns)


result = data_df.groupBy("category").agg(
    sum("sales").alias("total_sales"),
    avg("sales").alias("average_sales")
)


result.show()

result = data_df.groupBy("subcategory").agg(
    sum("sales").alias("total_sales"),
    avg("sales").alias("average_sales")
)

result.show()

+-----------+-----------+-----------------+
|   category|total_sales|    average_sales|
+-----------+-----------+-----------------+
|Electronics|       2000|666.6666666666666|
|   Clothing|        300|            150.0|
|  Furniture|        450|            225.0|
+-----------+-----------+-----------------+

+-----------+-----------+-------------+
|subcategory|total_sales|average_sales|
+-----------+-----------+-------------+
|     Laptop|       1200|       1200.0|
|     Mobile|        500|        500.0|
|     Tablet|        300|        300.0|
|      Chair|        150|        150.0|
|      Pants|        200|        200.0|
|      Table|        300|        300.0|
|     Shirts|        100|        100.0|
+-----------+-----------+-------------+



#Question 15

In [19]:
data = [
    (1, "ORD001", 5000),
    (2, "ORD002", 4000),
    (3, "ORD003", 7000),
    (1, "ORD004", 6000),
    (4, "ORD005", 8000),
    (5, "ORD006", 2000),
    (3, "ORD007", 5000)
]


columns = ["customer_id", "order_id", "order_amount"]


data_df = spark.createDataFrame(data, columns)


result = data_df.groupBy("customer_id").agg(sum("order_amount").alias("total_order_amount"))


result = result.filter(result.total_order_amount > 10000)


result.show()

+-----------+------------------+
|customer_id|total_order_amount|
+-----------+------------------+
|          1|             11000|
|          3|             12000|
+-----------+------------------+



#Question 16

In [20]:
data = [
    ("R1", "P1", "C1"),
    ("R2", "P1", "C2"),
    ("R3", "P2", "C3"),
    ("R4", "P2", "C1"),
    ("R5", "P3", "C4"),
    ("R6", "P3", "C5"),
    ("R7", "P3", "C4"),
    ("R8", "P1", "C3")
]


columns = ["review_id", "product_id", "customer_id"]


data_df = spark.createDataFrame(data, columns)


result = data_df.groupBy("product_id").agg(countDistinct("customer_id").alias("unique_reviewers"))


result.show()

+----------+----------------+
|product_id|unique_reviewers|
+----------+----------------+
|        P2|               2|
|        P3|               2|
|        P1|               3|
+----------+----------------+



#Question 17

In [21]:
data = [
    (1, "ProductA", 500),
    (1, "ProductB", 300),
    (1, "ProductA", 200),
    (2, "ProductA", 400),
    (2, "ProductC", 700),
    (3, "ProductB", 600),
    (3, "ProductC", 800)
]


columns = ["store_id", "product_id", "sales"]


data_df = spark.createDataFrame(data, columns)


window = Window.partitionBy("store_id", "product_id")


result = data_df.withColumn("total_sales", sum("sales").over(window))


result = result.select("store_id", "product_id", "total_sales").distinct()


result.show()

+--------+----------+-----------+
|store_id|product_id|total_sales|
+--------+----------+-----------+
|       1|  ProductA|        700|
|       1|  ProductB|        300|
|       2|  ProductA|        400|
|       2|  ProductC|        700|
|       3|  ProductB|        600|
|       3|  ProductC|        800|
+--------+----------+-----------+



#Question 18

In [22]:
data = [
    ("HR", 1, "john.doe@gmail.com"),
    ("IT", 2, "alice.smith@gmail.com"),
    ("HR", 3, "bob.jones@yahoo.com"),
    ("IT", 4, "charlie.brown@gmail.com"),
    ("Finance", 5, "dave.martin@gmail.com"),
    ("Finance", 6, "ellen.white@hotmail.com"),
    ("HR", 7, "george.king@gmail.com"),
]


columns = ["department", "employee_id", "email"]


data_df = spark.createDataFrame(data, columns)


result = data_df.filter(col("email").like("%@gmail.com%"))


result = result.groupBy("department").count().alias("gmail_count")


result.show()

+----------+-----+
|department|count|
+----------+-----+
|        HR|    2|
|        IT|    2|
|   Finance|    1|
+----------+-----+



#Question 19

In [23]:
data = [
    (1, "HR", 85),
    (2, "IT", 92),
    (3, "HR", 78),
    (4, "Finance", 95),
    (5, "IT", 88),
    (6, "Finance", 91),
    (7, "HR", 90)
]


columns = ["employee_id", "department", "score"]


data_df = spark.createDataFrame(data, columns)


result = data_df.groupBy("department").agg(max("score").alias("max_score"))


result = result.withColumnRenamed("department", "dept_max_score")


final_df = data_df.join(result, (data_df.department == result.dept_max_score) & (data_df.score == result.max_score)) \
                   .select(data_df["department"], data_df["employee_id"], data_df["score"])


final_df.show()


+----------+-----------+-----+
|department|employee_id|score|
+----------+-----------+-----+
|        IT|          2|   92|
|   Finance|          4|   95|
|        HR|          7|   90|
+----------+-----------+-----+



#Question 20

In [24]:
data = [
    (1, "Completed", 100),
    (2, "Pending", 200),
    (3, "Completed", 150),
    (4, "Pending", 250),
    (5, "Completed", 300),
    (6, "Completed", 50),
    (7, "Pending", 100),
]


columns = ["order_id", "order_status", "amount"]


df = spark.createDataFrame(data, columns)


result_df = df.groupBy("order_status").agg(
    sum("amount").alias("total_amount"),
    count(when(df["order_status"] == "Pending", 1)).alias("pending_count")
)


result_df.show()

+------------+------------+-------------+
|order_status|total_amount|pending_count|
+------------+------------+-------------+
|   Completed|         600|            0|
|     Pending|         550|            3|
+------------+------------+-------------+



#Some Other useful Functions

In [25]:
#from pyspark.sql.functions import to_date

#df.write.csv("data.csv", header=True)

#pandas_df = spark_df.toPandas()

#data = pd_df.values.tolist()

#df = df.withColumn("Price", regexp_replace(col("Price(Rs)"), "[^0-9.]", "").cast("double"))

#df = df.withColumn("date", to_date(col("date"), "yyyy-MM-dd"))

#df = df.dropna(subset=["critical column"])

#df = df.dropDuplicates(["order_id", "customer_id"])

#df = df.withColumn("product_name", regexp_replace(col("product_name"), "^\\d+\\. ", ""))

#df = df.withColumn("date", col("date").cast("string"))

#df = df.filter(col("quantity") > 0)

#df = df.select("order_id", "date")